In [1]:
import sys
sys.path
import os

base_path = os.path.abspath(".")

In [2]:
sys.path.append(os.path.join(base_path, 'src'))

In [7]:
import ato_tools

In [26]:
if 'ato_tools' in sys.modules:
    import importlib
    importlib.reload(scoring)

In [27]:
ato_tools.scoring.score("baseline.gdb", out_table="auto_baseline", mode = "Driving", test = True)

Scores written to baseline.gdb\auto_baseline_summary
Network ATO: 7453289.0
